# Masterlist overview

This notebook summarises some important properties of the data presented here. This includes

- The area covered by each field
- The number of total sources in each field
- The number of bands which we have data for in each field
- The size of the masterlist in each field

In [1]:
from astropy.table import Table, Column
from astropy import units as u
import numpy as np

from pymoc import MOC

from herschelhelp_internal.masterlist import find_last_ml_suffix

import os
import humanfriendly

## The definition of HELP DR1
Here we define DR1, which should be considered the principle definition in terms of the dated masterlist file that defines the object list on a given field. In some cases it is different to the masterlist used by an intermediate step because it was in progress as that step began. Nevertheless these files define DR1 so any discrepencies will force these objects lists through. The most likely upshot if this is that there may be objects in the dr1 masterlist that were not used in further steps and therefore lack, photozs, fir fluxes, or Cigale fits.

In [2]:
dr1 = [
# Field                suffix
['AKARI-NEP',          '20180215' ],
['AKARI-SEP',          '20180221' ],
['Bootes',             '20180227' ],
['CDFS-SWIRE',         '20180221' ],
['COSMOS',             '20180217' ],
['EGS',                '20180217' ],
['ELAIS-N1',           '20171016' ],
['ELAIS-N2',           '20180218' ],
['ELAIS-S1',           '20180221' ],
['GAMA-09',            '20171206' ],
['GAMA-12',            '20180218' ],
['GAMA-15',            '20180213' ],
['HDF-N',              '20180219' ],
['Herschel-Stripe-82', '20180301' ],
['Lockman-SWIRE',      '20180219' ],
['NGP',                '20180219' ],
['SA13',               '20180219' ],
['SGP',                '20180221' ],
['SPIRE-NEP',          '20180220' ],
['SSDF',               '20180221' ],
['xFLS',               '20180221' ],
['XMM-13hr',           '20180221' ],
['XMM-LSS',            '20180221' ]]

In [3]:
masterlist_summary = Table()
masterlist_summary.add_column(Column(data = np.array(dr1)[:,0], name = 'field'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), '', dtype="S20"), name = 'dr1_suffix'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), '', dtype="S20"), name = 'last_suffix'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), 0, dtype=int), name = 'objects'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), np.nan, dtype=float), name = 'area_sq_degrees'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), 0, dtype=int), name = 'file_size_bytes'))
masterlist_summary.add_column(Column(data =np.full(len(masterlist_summary), '', dtype="S20"), name = 'file_size_readable'))

In [4]:
help_moc = MOC(filename='../dmu2/help_coverage_MOC.fits')
print('Total HELP area: {} sqare degrees'.format(round(help_moc.area_sq_deg)))

Total HELP area: 1270 sqare degrees


In [5]:
for field in dr1:
    number_objects = len(Table.read('dmu1_ml_{}/data/master_list_cross_ident_{}_{}.fits'.format(field[0], 
                                                                                      field[0].lower(), 
                                                                                      field[1])))
    masterlist_summary['objects'][masterlist_summary['field'] == field[0]] = number_objects
    
    
    moc = MOC(filename='../dmu2/dmu2_field_coverages/{}_MOC.fits'.format(field[0]))
    area = round(moc.area_sq_deg)
    masterlist_summary['area_sq_degrees'][masterlist_summary['field'] == field[0]] = area
    
    masterlist_summary['dr1_suffix'][masterlist_summary['field'] == field[0]] = field[1]
    masterlist_summary['last_suffix'][masterlist_summary['field'] == field[0]] = find_last_ml_suffix('dmu1_ml_{}/data/'.format(field[0]))
    
    size = os.stat('dmu1_ml_{}/data/master_catalogue_{}_{}.fits'.format(field[0], 
                                                                                      field[0].lower(), 
                                                                                      field[1])).st_size

    
    
    #Can't install packages on Vulcain
    #size_readable = humanfriendly.format_size(size)
    masterlist_summary['file_size_bytes'][masterlist_summary['field'] == field[0]] = size
    masterlist_summary['file_size_readable'][masterlist_summary['field'] == field[0]] = humanfriendly.format_size(size)
    

    
    #TODO:
    num_surveys = 0.
    
    #Mean depth in g
    mean_depth = 0.
    
    num_bands = 0.

In [6]:
masterlist_summary.show_in_notebook()

idx,field,dr1_suffix,last_suffix,objects,area_sq_degrees,file_size_bytes,file_size_readable
0,AKARI-NEP,20180215,20180215,531746,9.0,348848640,348.85 MB
1,AKARI-SEP,20180221,20180221,844172,9.0,538606080,538.61 MB
2,Bootes,20180216,20180227,3367490,11.0,4310429760,4.31 GB
3,CDFS-SWIRE,20180221,20180221,2171051,13.0,5705593920,5.71 GB
4,COSMOS,20180217,20180217,2599374,5.0,10441782720,10.44 GB
5,EGS,20180217,20180217,1412613,4.0,3778816320,3.78 GB
6,ELAIS-N1,20171016,20180216,4026292,14.0,5242279680,5.24 GB
7,ELAIS-N2,20180218,20180218,1783240,9.0,2007964800,2.01 GB
8,ELAIS-S1,20180221,20180221,1655564,9.0,1970156160,1.97 GB
9,GAMA-09,20171206,20171206,12937982,62.0,19652852160,19.65 GB


In [7]:
print('Total number of objects in HELP: {}'.format(np.sum(masterlist_summary['objects'])))

Total number of objects in HELP: 171570436


In [8]:
print('Total size of masterlist files: {}'.format(humanfriendly.format_size(np.sum(masterlist_summary['file_size_bytes']))))

Total size of masterlist files: 304.93 GB


In [9]:
masterlist_summary.write('dr1_overview.fits', overwrite=True)